<a href="https://colab.research.google.com/github/Mohitduklan/Deep-Learning/blob/master/keras_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import pandas as pd
import numpy as np

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')



In [ ]:
dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
train_dir = os.path.join(dataset_dir, 'train')

['train', 'imdbEr.txt', 'test', 'README', 'imdb.vocab']

In [ ]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', 
    seed=seed)

Found 18119 files belonging to 2 classes.
Using 14496 files for training.


In [ ]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(3):
    print("Review", text_batch.numpy()[i])
    print("Label", label_batch.numpy()[i])

Review b"I'll come clean. The only reason I even found out about this DVD was because Dominic Monaghan is a favorite actor of mine. When I heard the title of the film, I thought it was going to be...different, perhaps in not such a good way.<br /><br />But I was wrong. After reading what few reviews were out there about this short, I was actually excited about seeing it. I sent off for my copy as soon as able and received it a few weeks later. Needless to say, I was not disappointed.<br /><br />The film follows Jack, a insomniac who is often plagued by conditions which causes him to doubt what is reality, and what is all in his head. I won't give away what happens, but I will tell you that the film can sometimes be frightening in it's realism.<br /><br />The directing is fantastic, focusing on what is essential to the story without allowing it to lose any entertainment or thought-provoking moments.<br /><br />All in all, I give this great film 9 out of 10, for going far beyond what I t

In [ ]:
print("Label 0 corresponds to", raw_train_ds.class_names[0])
print("Label 1 corresponds to", raw_train_ds.class_names[1])

Label 0 corresponds to neg
Label 1 corresponds to pos


In [ ]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html,
                                    '[%s]' % re.escape(string.punctuation),
                                    '')

In [ ]:
max_features = 10000
sequence_length = 250

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [ ]:
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [ ]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [ ]:
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Review tf.Tensor(b"If you know the story of Grey Owl, you'll love the movie! Annie Galipeau is a great actress, and Pierce is better than never in Grey Owl. But in this movie there's no real scene of action. I think this movie should be nominated at the Oscars! Welll go see this movie, it's A CLASSIC!", shape=(), dtype=string)
Label pos
Vectorized review (<tf.Tensor: shape=(1, 250), dtype=int64, numpy=
array([[  45,   24,  119,    2,   55,    5, 3056, 7806,  486,  102,    2,
          18, 3793,    1,    7,    4,   71,  513,    3, 4047,    7,  130,
          74,  113,    8, 3056, 7806,   17,    8,   11,   18,  226,   66,
         145,  131,    5,  215,   10,  103,   11,   18,  144,   28, 2142,
          32,    2, 2826,    1,  140,   63,   11,   18,   29,    4,  314,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
  

In [ ]:
print("1287 ---> ",vectorize_layer.get_vocabulary()[1287])
print(" 313 ---> ",vectorize_layer.get_vocabulary()[313])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

1287 --->  costumes
 313 --->  left
Vocabulary size: 10000


In [ ]:
train_ds = raw_train_ds.map(vectorize_text)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
embedding_dim = 16

model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160016    
_________________________________________________________________
dropout (Dropout)            (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 160,033
Trainable params: 160,033
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [ ]:
epochs = 10
history = model.fit(
    train_ds,
    epochs=epochs)

Epoch 1/10
453/453 [==============================] - 8s 16ms/step - loss: 0.6433 - binary_accuracy: 0.6788
Epoch 2/10
453/453 [==============================] - 3s 7ms/step - loss: 0.5817 - binary_accuracy: 0.6949
Epoch 3/10
453/453 [==============================] - 3s 6ms/step - loss: 0.5302 - binary_accuracy: 0.7115
Epoch 4/10
453/453 [==============================] - 3s 7ms/step - loss: 0.4652 - binary_accuracy: 0.7788
Epoch 5/10
453/453 [==============================] - 3s 6ms/step - loss: 0.4048 - binary_accuracy: 0.8305
Epoch 6/10
453/453 [==============================] - 3s 6ms/step - loss: 0.3586 - binary_accuracy: 0.8590
Epoch 7/10
453/453 [==============================] - 3s 6ms/step - loss: 0.3234 - binary_accuracy: 0.8751
Epoch 8/10
453/453 [==============================] - 3s 6ms/step - loss: 0.2985 - binary_accuracy: 0.8895
Epoch 9/10
453/453 [==============================] - 3s 7ms/step - loss: 0.2751 - binary_accuracy: 0.8995
Epoch 10/10
453/453 [===============